# Atividade Prática: Fluxos de Carbono e o sistema carbonato
**Instituto de Oceanografia - Interação Oceano-Atmosfera**
***
**Author:** [Andrés Piñango](https://github.com/andresawa/)  
Laboratório de Estudos dos Oceanos e Clima – LEOC, Instituto de Oceanografia, Universidade Federal do Rio Grande.  
email: andreseloy@furg.br  
**Last change:** 03/09/2021
***

## O fluxo de CO₂ na interface oceano-atmosfera
A troca de gases entre o oceano e a atmosfera é um processo importante no ciclo biogeoquímico de muitos elementos, como o CO₂ ou O₂. No caso do CO₂, a absorção de grandes quantidades deste gás proveniente das atividades antropogênicas pode alterar o equilíbrio do sistema carbonato do oceano, reduzindo a concentração do íon carbonato (e o OH⁻) e aumentando a concentração de prótons, em um processo conhecido como acidificação do oceano [(Doney et al., 2009)](https://doi.org/10.5670/oceanog.2009.93). Para compreender melhor essas alterações, é preciso conhecer quanto CO₂ os oceanos absorvem ou emitem num determinado período. Esse balanço entre emissão/absorção é chamado de fluxo líquido (FCO₂), e ele descreve o comportamento do oceano com relação à captação ou liberação de CO₂ atmosférico.  

Para estimar o fluxo de um gás através da interface ar-água, é necessario conhecer dois informações:
1) O desequilíbrio entre as concentrações do gás no ar e na superfície do oceano (controla a direção do fluxo)  
2) A taxa em que o desequilíbrio é removido (controla a intensidade do fluxo) 

Então, os fluxos de CO₂ em escalas regionais a globais são geralmente determinados usando a seguinte equação:  

$$
FCO_{2} = k \; K_{0} \; \left ( pCO_{2 \; oceano} - pCO_{2 \; ar} \right )
$$

Onde:
* $FCO_{2}$ é o fluxo de CO₂ (massa área⁻¹ tempo⁻¹)
* $k$ é o efeito da velocidade do vento na troca gasosa (comprimento tempo⁻¹)
* $K_{0}$ é a solubilidade do gás (massa volume⁻¹ pressão⁻¹)
* $pCO_{2}$ é a pressão parcial do CO₂ no mar e atmosfera

Em Python, $FCO_{2}$ pode ser calculado usando a seguinte função:

In [ ]:
def carbon_flux(gastransfer, solubility, fco2_sw, fco2_atm):
    result = gastransfer * solubility * (fco2_sw - fco2_atm)
    return result

Uma explicação mais detalhada dos parâmetros da equação do fluxo é apresentada abaixo.

### Efeito do vento: o coeficiente de transferência gasosa (*k*)
O aumento da velocidade do vento permite uma agitação mais vigorosa da superfície oceânica, aumentando a área de superfície, a formação de bolhas e outros processos, incrementando a troca de gases. A relação entre a velocidade do vento e o fluxo de gás é empírica e existem várias formulações. Uma das mais utilizadas é a relação descrita por [Wanninkhof (2014)](https://doi.org/10.4319/lom.2014.12.351):  

$$
k = 0.251\; U^{2}\;  (Sc/660)^{-0.5}
$$

Onde: 
* $U$ é a velocidade do vento a uma altura de 10 metros acima da superfície oceânica.
* $Sc$ é o número de Schmidt, uma relação entre a taxa de difusão molecular de um gás em um fluido e a viscosidade desse fluido.
* $k$ é o efeito da velocidade do vento na troca gasosa (cm h⁻¹)  


> Fun fact: a parametrização de [Ho *et al.* (2006)](https://doi.org/10.1029/2006GL026817) é equivalente à parametrização de [Wanninkhof (2014)](https://doi.org/10.4319/lom.2014.12.351) e muito mais fácil de pronunciar 😆. Recomendo a leitura [desta](https://twitter.com/_david_ho_/status/1364846020778102788) sequência no Twitter.


O número de Schmidt (*Sc*) depende de cada gás, e pode ser calculado para água salgada (salinidade = 35) desde –2°C até 40°C usando a equação descrita em [Wanninkhof (2014)]( https://doi.org/10.4319/lom.2014.12.351): 

$$
Sc = A + Bt + Ct^{2} + Dt^{3} + Et^{4}
$$

Onde $t$ é a temperatura (°C), é os coeficientes para o CO₂ são:  

| Gás | A       | B       | C      | D         | E          |
|-----|---------|---------|--------|-----------|------------|
| CO₂ | 21116.8 | -136.25 | 4.7353 | -0.092307 | 0.00075555 |

Em Python, $k$ (em (m dia⁻¹) pode ser calculado usando a seguinte função:

In [ ]:
def gastransfer_wanninkhof(temperature, wind_speed):
    A = 2116.8
    B = -136.25
    C = 4.7353
    D = -0.092307
    E = 0.0007555
    Sc = A + B * temperature + C * temperature**2 + D * temperature**3 + E * temperature**4
    result = (0.251 * wind_speed**2 * (Sc / 660)**-0.5) * 24/100 #24/100 tranformar de (cm h⁻¹) a (m dia⁻¹)
    return result

### O coeficiente de solubilidade do gás (*K₀*)

A solubilidade dum gás num liquido é proporcional á pressão parcial do mesmo, seguindo a [Lei de Henry](https://en.wikipedia.org/wiki/Henry%27s_law). As solubilidades dos gáses dependem fortemente da temperatura e, em geral, eles são menos solúveis em temperaturas mais altas. A salinidade tambem tem um efeito na solubilidade dos gases, e geralmente baixas solubilidades encuentra-se associadas a salinidades altas. A solubilidade do CO₂ e outros gáses no oceano pode ser calculada mediante a equação descrita em [Weiss (1974)](https://doi.org/10.1016/0304-4203(74)90015-2): 

$$
K_{0} = e^{A1 +A2 \frac{100}{T} + A3\, ln \left ( \frac{T}{100} \right ) + S \left [B1 + B2 \frac{T}{100} + B3  (\frac{T}{100})^{2} \right ]}
$$

Onde $T$ é a temperatura (em Kelvin), $S$ é a salinidade é os coeficientes para o CO₂ são:  

| Gás | A1       | A2      | A3      | B1       | B2        | B3        |
|-----|----------|---------|---------|----------|-----------|-----------|
| CO₂ | -58.0931 | 90.5069 | 22.2940 | 0.027766 | -0.025888 | 0.0050578 |

Em Python, $K_{0}$ pode ser calculado usando a seguinte função:

In [ ]:
def solubility_weiss(temperature, salinity):
    A1 = -58.0931
    A2 = 90.5069
    A3 = 22.2940
    B1 = 0.027766
    B2 = -0.025888
    B3 = 0.0050578
    tempK = temperature + 273.15 # °C a K
    result =  np.exp(
        A1 + 
        A2 * (100/tempK) + 
        A3 * np.log(tempK/100) + 
        salinity * (B1 + 
                    B2 * (tempK/100) + 
                    B3 * (tempK/100)**2
                   )
    )
    return result

***
## Atividade 1. Calculo do fluxo de CO₂ em duas regiões do Oceano Atlântico

**Objetivo:** Calcular o fluxo líquido de CO₂ entre o oceano e a atmosfera (FCO₂) em duas regiões do Oceano Atlãntico (região subtropical norte e região tropical) desde o ano 2000 até o 2020 e identificar se essas regiões são uma fonte ou um sumidouro de CO₂ atmosférico.  

**Procedimento:** Os alunos devem utilizar as equações apresentadas acima para estimar o fluxo de CO₂ (em $\frac{mmol}{m^{2}\, dia}$) nos seguintes pontos:

|  Ponto  | Temperatura  (°C) | Salinidade | fCO₂ oceano (μatm) | Velocidade Vento (m/s) | pCO₂ atmosfera (μatm) |
|:-------:|:-----------------:|:----------:|:------------------:|:----------------------:|:---------------------:|
| Ponto 1 |       17.27       |    35.65   |       381.13       |          5.77          |         413.25        |
| Ponto 2 |       -0.59       |    33.27   |       326.74       |          4.83          |         401.08        |

Os resultados deven ser inseridos na seguinte celda, que irá validar se a resposta é correta. Se o cálculo estiver correto, os alunos poderão usar o código em anexo que calculará o fluxo de CO₂ durante os últimos 20 anos automaticamente para as duas regiões avaliadas no oceano Atlântico. Uma descrição detalhada de como os dados foram preparados, incluindo as fontes utilizadas, pode ser encontrada aqui.

**Resultados Esperados**: Neste trabalho, espera-se que o aluno entenda como o fluxo de CO₂ entre a interface ar-mar é calculado a partir dos dados fornecidos. Além disso, é esperado que o aluno explique os resultados encontrados de forma crítica, considerando todo o conhecimento adquirido nas últimas aulas de Interação Oceano Atmosfera. Tente responder especialmente estas perguntas:

* O fluxo de CO₂ é semelhante nas duas regiões avaliadas? Se existir diferenças, que poder estar ocasionando isso?
* O fluxo de CO₂ calculado concorda com a literatura/trabalhos previos nessas regiões?

Isso será demonstrado através do relatório que o aluno fará, abordando uma breve descripção das área de estudo, e a discusão dos resultados encontrados.

#### Inserir os fluxo de CO₂ calculados por você aqui:

In [ ]:
Fco2_ponto1 = 0
Fco2_ponto2 = 0
exec(open("test.py").read())

#### Se o resultado é correto, você pode usar este codigo para calcular os fluxos nas duas regiões desde o 2000 até o 2020

In [ ]:
# Importa as librerias
import numpy as np
import pandas as pd
import xarray as xr

# Importa os dados das duas regiões
if (teste):
    data_north = xr.open_dataset("north_data.nc")
    data_equator = xr.open_dataset("equator_data.nc")

# Calcula o efeito da velocidade do vento no fluxo de CO₂ (k)
k_north = xr.apply_ufunc(gastransfer_wanninkhof, data_north.temperature, data_north.wind_speed)
k_equator = xr.apply_ufunc(gastransfer_wanninkhof, data_equator.temperature, data_equator.wind_speed)

# Calcula o coeficiente de solubilidade (K₀)
K0_north = xr.apply_ufunc(solubility_weiss, data_north.temperature, data_north.salinity)
K0_equator = xr.apply_ufunc(solubility_weiss, data_equator.temperature, data_equator.salinity)

# Calcula os fluxos de CO₂ nas duas regiões (FCO₂)
fluxo_north = xr.apply_ufunc(carbon_flux, k_north, K0_north, data_north.fco2, data_north.pco2_atm)
fluxo_equator = xr.apply_ufunc(carbon_flux, k_equator, K0_equator, data_equator.fco2, data_equator.pco2_atm)

# Calcula o valor medio de FCO2 na região para todos os meses
Fco2_north = fluxo_north.mean(["latitude", "longitude"])
Fco2_equator = fluxo_equator.mean(["latitude", "longitude"])

# Calcular o valor medio para todo o periodo estudado
mean_north = Fco2_north.mean()
mean_equator = Fco2_equator.mean()

***
## Atividade 2. O sistema carbonato: calculo do pH a partir de pCO₂ e alcalinidade

**Objetivo:** Calcular o pH das águas superficias a partir dos valores de pCO₂ e alcalinidade nas duas regiões do Oceano Atlãntico estudadas, desde o ano 2000 até o 2020, e identificar se existen variações neste parametro.  

**Procedimento:** Os parâmetros do sistema carbonato podem ser calculados a partir de dois variáveis (pCO₂, pH, DIC ou TA) e dados de temperatura, salinidade e pressão (ou profundidade), utilizando diversos pacotes. Neste exercicio, os alunos deven calcular a alcalinidade potencial das aguas superficiais (alcalininade potencial = alcalinidade total + concentração nitrato) mediante as relações calculadas por [Takahashi et al. (2014)](https://doi.org/10.1016/j.marchem.2014.06.004), e depois, a partir da alcalinidade total calculada e os dados de fCO₂ disponiveis, calcular o pH mediante o pacote [PyCO2SYS](10.5281/zenodo.5176106). O codigo para fazer todos os calculos encuentra-se na seguinte celda. As relações de Takahashi para as regiões estudadas, assím como os valores medios de nitrato provenientes do [World Ocean Atlas 2018](https://www.ncei.noaa.gov/products/world-ocean-atlas) podem ser encontrados na seguinte tabela:

|               Região              | Relação Alcalinidade Potencial - Salinidade | Concentração Nitrato (μmol/kg) |
|:---------------------------------:|:-------------------------------------------:|:------------------------------:|
|    Atlântico Norte (40°N-60°N)    |     PALK = (45.30 x Salinidade) + 733.0     |               4.4              |
| Altlântico Equatorial (10°S-10°N) |     PALK = (58.25 x Salinidade) + 270.9     |               0.3              |

**Resultados Esperados**: Neste trabalho, espera-se que o aluno entenda como os parâmetros do sistema carbonato (exemplo: o pH) é calculado a partir dos dados fornecidos. Além disso, é esperado que o aluno explique os resultados encontrados de forma crítica, considerando todo o conhecimento adquirido nas últimas aulas de Interação Oceano Atmosfera. Tente responder especialmente estas perguntas:

* Qual é a tendencia do pH no tempo?
* As variações do pH no tempo são semelhante nas duas regiões avaliadas? Se existir diferenças, que poder estar ocasionando isso?
* As taxas de variacão do pH encontradas concordam com a literatura/trabalhos previos nessas regiões?

Isso será demonstrado no mesmo relatorio da Atividade 1. 

In [ ]:
# Importa as librerias
import PyCO2SYS as pyco2

# Função para calcular a alcalinidade (Alk) a partir da relação alcalinidate potencial (PAlk) - Salinidade e a concentração de nitrato
def alkalinity(salinity, region):
    if (region == "north_atlantic"):
        palk = (45.30 * salinity) + 733.0
        result = palk - 4.4
        return result
    elif (region == "equatorial_atlantic"):
        palk = (58.25 * salinity) + 270.9
        result = palk - 0.3
        return result
    else:
        print('A região introducida não é correta, os possíveis valores são "north_atlantic" ou "equatorial_atlantic". Por favor tente novamente.')

# Calcula a alcalinidade (TA) para as duas regiões
alk_north = xr.apply_ufunc(alkalinity, data_north.salinity, "north_atlantic")
alk_equator = xr.apply_ufunc(alkalinity, data_equator.salinity, "equatorial_atlantic")

# Função para calcular o pH
def pH_calc(alkalinity_sw, fco2_sw, temperature_sw, salinity_sw):
    carbonate_system = pyco2.sys(par1 = alkalinity_sw, 
                                 par2 = fco2_sw, 
                                 par1_type = 1, 
                                 par2_type = 5, 
                                 salinity = salinity_sw,
                                 temperature = temperature_sw,
                                 opt_gas_constant = 1
                                )
    result = carbonate_system["pH_total"]
    return result

# Calcula o pH
pH_north = xr.apply_ufunc(pH_calc, alk_north, data_north.fco2, data_north.temperature, data_north.salinity)
pH_equator = xr.apply_ufunc(pH_calc, alk_equator, data_equator.fco2, data_equator.temperature, data_equator.salinity)

# Calcula o valor medio para as duas regiões
mpH_north = pH_north.mean(["latitude", "longitude"])
mpH_equator = pH_equator.mean(["latitude", "longitude"])

# Calcula o valor medio de FCO2 na região agrupado por mes
monthly_pH_north = pH_north.groupby("time.month").mean(["latitude", "longitude", "time"])
monthly_pH_equator = pH_equator.groupby("time.month").mean(["latitude", "longitude", "time"])

# Calcula a anomalia de FCO2 para todos os meses
apH_north = mpH_north.groupby("time.month") - monthly_pH_north
apH_equator = mpH_equator.groupby("time.month") - monthly_pH_equator

# Calcula a taxa (trend) da acidificação com as anomalias
idx_north = np.isfinite(mpH_north)
trend_north = np.polyfit(np.arange(0, apH_north.shape[0])[idx_north], apH_north.values[idx_north], 1)
idx_equator = np.isfinite(mpH_equator)
trend_equator = np.polyfit(np.arange(0, apH_equator.shape[0])[idx_equator], apH_equator.values[idx_equator], 1)

***
## Resultados

In [ ]:
# Importar a libreria para as figuras
import matplotlib.pyplot as plt

# Graficar a serie temporal dos fluxos de CO₂
fig, ax = plt.subplots(3, 1, figsize=(10, 8), dpi = 300)

# Show the first subplot
ax[0].plot(Fco2_north.time, Fco2_north.values, c='r')
ax[0].axhline(mean_north, c = "r", alpha = 0.3, linestyle = '--', label = "Média: -4.17") #mean_north
ax[0].plot(Fco2_equator.time, Fco2_equator.values, c='b')
ax[0].axhline(mean_equator, c = "b", alpha = 0.3, linestyle = '--', label = "Média: 0.30") #mean_equator
ax[0].axhline(0, c = "black", alpha = 0.7)
ax[0].set_ylabel("FCO₂ [mmol m⁻² day⁻¹]")
ax[0].legend()

# Show the second subplot
ax[1].plot(data_north.time, data_north.fco2.mean(["latitude", "longitude"]), c='r', label = "Atlântico Norte")
ax[1].plot(data_equator.time, data_equator.fco2.mean(["latitude", "longitude"]), c='b',  label = "Atlântico Tropical")
ax[1].plot(data_equator.sel(time = slice("2001", "2020")).time, 
           data_equator.sel(time = slice("2001", "2020")).pco2_atm, c='black', alpha = 0.7,  label = "Atmosfera")
ax[1].set_ylabel("pCO₂ [μatm]");
ax[1].legend()

# Show the third subplot
ax[2].plot(mpH_north.time, mpH_north.values, c='r')
ax[2].plot(mpH_equator.time, mpH_equator.values, c='b')
ax[2].plot(mpH_north.time[12:], 
           (np.arange(12, apH_north.shape[0]) * trend_north[0]) + 8.11, 
           c='r', alpha = 0.3, linestyle = '--', label = "Taxa de acidificação: -0.0016 ano⁻¹") #trend_north[0] * 12
ax[2].plot(mpH_equator.time[12:], 
           (np.arange(12, apH_equator.shape[0]) * trend_equator[0]) + 8.07, 
           c='b', alpha = 0.3, linestyle = '--', label = "Taxa de acidificação: -0.0010 ano⁻¹") #trend_equator[0] * 12
ax[2].set_ylabel("pH [total scale]")
ax[2].set_xlabel("Time [years]")
ax[2].legend();